In [1]:
from IPython.core.display import HTML
import pandas as pd
import numpy as np
from tableone_modded.tableone import TableOne
from utils import *

In [2]:
combined,var_list,cat_features_list,cat_order = get_data()
combined,var_list,cat_features_list,cat_order = apply_exclusions(combined, var_list,cat_features_list,cat_order,exclude_death=False)

C:\Users\Wesley\Documents\NUH Research\LV thrombus\utils.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tropi'] = df['tropi'].replace({999.:np.nan})
C:\Users\Wesley\Documents\NUH Research\LV thrombus\utils.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stenttype'] = df['stenttype'].replace({999.0:'4.0'})
C:\Users\Wesley\Documents\NUH Research\LV thrombus\utils.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

Cohort Size: 289
Processing dataset including patients who died...
No anticoagulation: 14
Unknown outcome: 31
Final cohort size:244



C:\Users\Wesley\Documents\NUH Research\LV thrombus\utils.py:345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['Peak Troponin I, ng/dL'] = combined['Peak Troponin I, ng/dL'].replace({999.:np.nan})
C:\Users\Wesley\Documents\NUH Research\LV thrombus\utils.py:346: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['lvtrecurrence'][~combined['lvtrecurrence'].isin(['0.0','1.0','2.0'])] = np.nan
C:\Users\Wesley\Anaconda3\envs\lv_thrombus\lib\site-packages\pandas\core\generic.py:8767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [3]:
mytable = TableOne(combined, var_list, categorical=cat_features_list, groupby='lvtstatus',order=cat_order,pval=True,remarks=False,reverse_missing=True)
HTML(mytable.tabulate(tablefmt="html"))

C:\Users\Wesley\Documents\NUH Research\LV thrombus\tableone_modded\tableone.py:762: UserWarning: Chi-squared test for {} may be invalid (expected cell counts are < 5).
  "(expected cell counts are < 5).".format(v))
C:\Users\Wesley\Anaconda3\envs\lv_thrombus\lib\site-packages\numpy\core\fromnumeric.py:3584: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\Wesley\Anaconda3\envs\lv_thrombus\lib\site-packages\numpy\core\_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Wesley\Anaconda3\envs\lv_thrombus\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Wesley\Anaconda3\envs\lv_thrombus\lib\site-packages\numpy\lib\nanfunctions.py:1667: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
C:\Users\Wesley\Anaconda3\envs\lv_thrombus\lib\site-packages\numpy\cor

,,Count,Resolved LVT,Unresolved LVT/Death,P-Value,Test
n,,,156,88,,
"Age, years",,244,58.4 (12.1),61.5 (14.3),0.090,Two Sample T-test
Sex,Male,244,134 (85.9),78 (88.6),0.681,Chi-squared
,Female,,22 (14.1),10 (11.4),,
"Height, cm",,190,1.7 (0.1),1.6 (0.1),0.168,Two Sample T-test
"Weight, kg",,220,70.4 (17.5),66.0 (14.1),0.045,Two Sample T-test
Ethnicity,Chinese,244,90 (57.7),37 (42.0),0.121,Chi-squared (warning: expected count < 5)
,Malay,,42 (26.9),34 (38.6),,
,Indian,,17 (10.9),13 (14.8),,
,Others,,7 (4.5),4 (4.5),,


In [4]:
mytable.to_excel('tableone.xlsx')

In [5]:
for status in ['All Cases','Resolved LVT','Unresolved LVT/Death']:
    if status == 'All Cases':
        series = combined['Followup Duration, days']
    else:
        series = combined['Followup Duration, days'][combined['lvtstatus'] == status]
    print(status)
    print(f'Median: {series.median()}')
    print(f'IQR: {series.quantile(0.25)}-{series.quantile(0.75)}')

All Cases
Median: 807.0
IQR: 264.75-1746.25
Resolved LVT
Median: 1049.0
IQR: 598.5-2163.5
Unresolved LVT/Death
Median: 196.5
IQR: 15.75-908.25


In [6]:
combined = combined.drop('Followup Duration, days',axis=1)
combined['lvtstatus'] = combined['lvtstatus'].replace({'Resolved LVT':0,'Unresolved LVT/Death':1})
combined['lvtstatus'].to_csv('processed_data\\outcome.csv',index=False)
predictors = combined.drop('lvtstatus',axis=1)
predictors.to_csv('processed_data\\predictors.csv',index=False)
pd.Series(cat_features_list).to_csv('processed_data\\categorical_features.csv',index=False)
pd.Series(list(set(list(predictors)) - set(cat_features_list))).to_csv('processed_data\\numeric_features.csv',index=False)